# Import

In [8]:
import time
import random
import math
from itertools import product
from collections import namedtuple
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

from scipy.optimize import linprog

import gym

# Frozen Lake
Frozen Lake is a good environment to demonstrate reinforcement learning methods in the tabular case.

In [9]:
env = gym.make('FrozenLake-v0')

print("Observation space : \t{}".format(env.observation_space))
print("Action space : \t\t{}".format(env.action_space))

Observation space : 	Discrete(16)
Action space : 		Discrete(4)


In [3]:
# Random agent
env.reset()
for _ in range(10):
    env.render()
    env.step(env.action_space.sample())
    clear_output(wait=True)
    time.sleep(0.1)
env.close()

  (Up)
SFFF
FHFH
FFFH
HFFG


A fundamental property of value functions used throughout reinforcement learning and dynamic programming is that they satisfy recursive relationships. For any policy $\pi$ and any state $s$, the following consistency condition holds between the value of $s$ and the value of its possible successor states:
$$v_\pi(s)=\sum\limits_{a}\pi(a|s) \sum\limits_{s', r} p(s',r|s,a) [r + \gamma v_\pi(s')]$$

The value function can be computed by solving this system of linear equations.

## Linear Programming
As we want to compute the value function for the optimal policy $\pi^*$ and not any policy $\pi$, let's rewrite the system of linear equations. The Bellman optimality equation states that
$$\forall s\in S, v_*(s)=\max\limits_{a}\sum\limits_{s', r} p(s',r|s,a) [r + \gamma v_*(s')]$$
Therefore
$$\forall (s, a) \in S\times A, \quad v_*(s) \geq \sum\limits_{s', r} p(s',r|s,a) [r + \gamma v_*(s')]$$
$$\forall (s, a) \in S\times A, \quad v_*(s) - \gamma \sum\limits_{s', r} p(s',r|s,a) v_*(s') \geq \sum\limits_{s', r} p(s',r|s,a) r$$

The optimal value function can then be found by solving a linear programming problem:
$$\left\{ \begin{array}{c}
\min \sum\limits_{s} v(s)\\
s.t. \ \forall (s,a)\in S\times A, \quad v(s) - \gamma \sum\limits_{s', r} p(s',r|s,a) v(s') \geq \sum\limits_{s', r} p(s',r|s,a) r
\end{array}\right.$$
Which, finally, is a linear program with $|S|$ variables and $|S||A|$ constraints, if the environment's dynamics are completely known.

In [6]:
gamma = 1

c = np.ones(env.observation_space.n)
A_ub = np.zeros((env.observation_space.n*env.action_space.n, env.observation_space.n))
b_ub = np.zeros((env.observation_space.n*env.action_space.n,))
for (i, (s, a)) in enumerate(product(range(env.observation_space.n), range(env.action_space.n))):
    r = sum([p*r for p, s_, r, d in env.P[s][a]])
    b_ub[i] = r
    
    A_ub[i][s] = 1
    for p, s_, r, d in env.P[s][a]:
        A_ub[i][s_] += -gamma*p
A_ub = -A_ub
b_ub = -b_ub

res = linprog(c, A_ub=A_ub, b_ub=b_ub)

/home/max/anaconda3/envs/rl/lib/python3.7/site-packages/ipykernel_launcher.py:17: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
/home/max/anaconda3/envs/rl/lib/python3.7/site-packages/ipykernel_launcher.py:17: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
/home/max/anaconda3/envs/rl/lib/python3.7/site-packages/scipy/optimize/_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=2.078e-18): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


In [7]:
print(res["message"])
V = res["x"].reshape(4, 4)
print(V)

Optimization terminated successfully.
[[8.23529412e-01 8.23529412e-01 8.23529412e-01 8.23529412e-01]
 [8.23529412e-01 1.26967246e-11 5.29411765e-01 5.56204911e-12]
 [8.23529412e-01 8.23529412e-01 7.64705882e-01 3.57139713e-12]
 [1.67169236e-11 8.82352941e-01 9.41176471e-01 2.97834464e-13]]


# Policy Iteration and Value Iteration

In [10]:
def policy_iteration(env, gamma, epsilon, V=np.zeros(env.observation_space.n), pi=np.zeros(env.observation_space.n)):
    while True:
        delta = 0
        for s in range(env.observation_space.n):
            v = V[s]
            V[s] = sum([p*(r + gamma*V[s_]) for p, s_, r, d in env.P[s][pi[s]]])
            delta = max(delta, abs(v-V[s]))
        if delta < epsilon:
            break
    
    policy_stable = True
    for s in range(env.observation_space.n):
        old_action = pi[s]
        pi[s] = np.argmax([sum([p*(r + gamma*V[s_]) for p, s_, r, d in env.P[s][a]]) for a in range(env.action_space.n)])
        if old_action != pi[s]:
            policy_stable = False
    if policy_stable == True:
        return V, pi
    else:
        return policy_iteration(env, gamma=gamma, epsilon=epsilon, V=V, pi=pi)

In [26]:
def value_iteration(env, gamma, epsilon):
    V = np.zeros(env.observation_space.n)
    pi = np.zeros(env.observation_space.n)
    while True:
        delta = 0
        for s in range(env.observation_space.n):
            v = V[s]
            update = [sum([p*(r + gamma*V[s_]) for p, s_, r, d in env.P[s][a]]) for a in env.P[s].keys()]
            V[s] = max(update)
            pi[s] = np.argmax(update)
            delta = max(delta, abs(v-V[s]))
        if delta < epsilon:
            return V, pi

In [25]:
V, pi = policy_iteration(env, gamma=1, epsilon=0.0001)
print(V.reshape(4, 4))

[[0.82188098 0.82133959 0.82096328 0.8207718 ]
 [0.82204676 0.         0.52828734 0.        ]
 [0.82230639 0.82263933 0.76392582 0.        ]
 [0.         0.88173433 0.9408617  0.        ]]


In [28]:
V, pi = value_iteration(env, gamma=1, epsilon=0.0001)
print(V.reshape(4, 4))

[[0.82182145 0.82126109 0.82087163 0.82067347]
 [0.82199325 0.         0.52824715 0.        ]
 [0.82226231 0.82260733 0.76389785 0.        ]
 [0.         0.88171208 0.94085038 0.        ]]
